## Load Libraries

In [1]:
import numpy as np

import jax
from numpyro.infer.autoguide import AutoNormal
from numpyro.infer.initialization import init_to_value

from aggpp.sim import generate_data, assign_overlapping_bins
from aggpp.models import OverlapAggPP

# Visualization
import altair as alt

## Generate Data

Generate data.

In [24]:
N = 1000
df_true, df_data = generate_data(N, M = 2, seed=42)

In [25]:

x = df_data['x'].values
left_limits = [0] + list(range(5, 95, 5))
right_limits = list(range(10, 101, 5))

sampling_effort = [1] * len(left_limits)
df_data['range'] = assign_overlapping_bins(x, left_limits, right_limits, sampling_effort)

tmp = df_data.groupby('range', observed=True).agg(
	y_agg = ('y', 'sum'),
	N = ('y', 'count')
).reset_index()

df_data = df_data.merge(tmp, on='range', how='left')
df_data['rate_agg'] = df_data['y_agg'] / df_data['N']
df_data.head(5)

,x,log_exposure,f,rate,y,range,y_agg,N,rate_agg
0,82,-1.0250,-0.836819,0.155390,0,"[80, 90)",16,51,0.313725
1,86,-1.0750,-0.056807,0.322450,1,"[80, 90)",16,51,0.313725
2,74,-0.9250,-0.315744,0.289169,0,"[70, 80)",12,37,0.324324
3,74,-0.9250,-0.315744,0.289169,0,"[65, 75)",76,47,1.617021
4,87,-1.0875,0.165866,0.397869,0,"[85, 95)",32,62,0.516129


In [26]:
df_agg = (
  df_data[['range', 'rate_agg']]
  .drop_duplicates()
  .sort_values('range')
  .reset_index(drop=True)
)

df_agg['x_left'] = df_agg['range'].apply(lambda x: x.left)
df_agg['x_right'] = df_agg['range'].apply(lambda x: x.right)

df_agg['x_left'] = df_agg['x_left'].astype(float)
df_agg['x_right'] = df_agg['x_right'].astype(float)

In [27]:
df_plot = df_data.copy()
df_plot.drop(columns=['range', 'y_agg', 'N'], inplace=True)
df_agg_plot = df_agg.drop(columns='range')

rate = (
	alt.Chart(df_plot).mark_line(color='#de425b').encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate', title='Rate'),
	)
)

count = alt.Chart(df_plot).mark_point(size=3).encode(
	x=alt.X('x', title='x'),
	y=alt.Y('y', title='Count'),
)

segment = alt.Chart(df_agg_plot).mark_rule(
  color='#ff6361',
  size=1.5
).encode(	
	x=alt.X('x_left', title='x'),  # Use same title and scale
	x2=alt.X2('x_right'),
	y=alt.Y('rate_agg', title='Rate'),
)

# Use resolve_scale to ensure all plots share the same x-axis
(rate + count + segment).resolve_scale(x='shared').properties(width=600, height=200)

alt.LayerChart(...)

In [28]:
overlap_model = OverlapAggPP(df_data, df_true, L = 5)

The following model should fail to accurately infer the rate.

In [29]:
prng_key = jax.random.PRNGKey(42)
init_values = {
	'baseline': -df_true['log_exposure'].values.mean()
}
guide = AutoNormal(overlap_model.model, init_loc_fn=init_to_value(values=init_values))
overlap_model.run_inference_svi(prng_key, guide)

100%|██████████| 5000/5000 [00:48<00:00, 102.99it/s, init loss: 12674.9219, avg. loss [4751-5000]: 464.8880]


In [30]:
post_pred = overlap_model.posterior_predictive_svi(prng_key, guide)
post_rate_sum = np.quantile(post_pred['rate'], q = (0.025, 0.5, 0.975), axis=0)

In [31]:
df_true['M'] = post_rate_sum[1][df_true['x'].values]
df_true['CL'] = post_rate_sum[0][df_true['x'].values]
df_true['CU'] = post_rate_sum[2][df_true['x'].values]

In [32]:
rate = (
	alt.Chart(df_true)
	.mark_line(color='red')
	.encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate', title='Intensity'),
	)
)

rate_est = (
	alt.Chart(df_true)
	.mark_line(color='green')  # Changed
	.encode(
		x=alt.X('x', title='x'),
		y=alt.Y('M', title='Intensity'),
	)
)

bands = alt.Chart(df_true).mark_errorband(color='green').encode(
	x=alt.X('x', title='x'),
	y=alt.Y('CL', title='Intensity'),
	y2=alt.Y2('CU', title='Intensity'),
)

(rate + rate_est + bands).properties(width=600, height=200)

alt.LayerChart(...)